In [ ]:
import scanpy as sc
import muon as mu
import numpy as np
import pandas as pd
import episcanpy as epi
import os
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
sc_file = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files'
mdata = mu.read(f'{sc_file}/multi_all_cells_raw.h5mu')
atac = mdata.mod['atac']
tf_bed = pd.read_csv("/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/230609_aggregate/outs/analysis/tf_analysis/peak_motif_mapping.bed",
                     sep='\t',
                     index_col=None,
                     header=None
                    )

In [ ]:
tf_bed

In [ ]:
# tf_bed[2] = tf_bed[2]-1 # end of peak name from cellranger-arc seems to be + 1


In [ ]:
atac.uns['atac']['peak_annotation']

In [ ]:
# tf_bed[2] = tf_bed[2]-1 # end of peak name from cellranger-arc seems to be + 1
tf_bed['peak_name'] = tf_bed[0] +":"+tf_bed[1].astype('string')+'-'+tf_bed[2].astype('string')
df = tf_bed.groupby('peak_name')[3].apply(lambda x: ','.join(x.astype(str))).reset_index()
peak_tf_dt = pd.Series(df[3].values,index=df.peak_name)
peak_tf_dt = pd.Series(df[3].values, index=df.peak_name)
atac.var['tfs'] = pd.Series(peak_tf_dt)
atac.var['tfs']

In [ ]:
atac.var_names

In [ ]:
peak_tf_dt['chr1:3119667-3120573']

In [ ]:
output_fol = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/pilot/230614_episcanpy_pilot'
os.makedirs(output_fol, exist_ok=True)
sc.set_figure_params(dpi_save = 300, fontsize=10)
sc.settings.figdir = output_fol
epi.settings.figdir = output_fol
epi.set_figure_params(dpi_save=300, fontsize=10)
plt.rcParams["font.family"] = "Arial"

In [ ]:
adata_rna = sc.read('/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/share/p7_multiome_rna.gz.h5ad')
sc.pp.normalize_total(adata_rna, target_sum=1e6)
sc.pp.log1p(adata_rna,base=10)

In [ ]:
adata_atac = sc.read('/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/share/p7_multiome_atac.gz.h5ad')

In [ ]:
tf_bed = pd.read_csv("/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/230609_aggregate/outs/analysis/tf_analysis/peak_motif_mapping.bed",
                     sep='\t',
                     index_col=None,
                     header=None
                    )


In [ ]:
len(no_tf)

In [ ]:
epi.pp.filter_cells(adata_atac, min_features=1)
epi.pp.filter_features(adata_atac, min_cells=1)
adata_atac.obs['log_nb_features'] = [np.log10(x) for x in adata_atac.obs['nb_features']]
sc.pl.violin(adata_atac, ['nb_features'], groupby='mouse', jitter=False,inner='box')
sc.pl.violin(adata_atac, ['log_nb_features'], groupby='mouse',jitter=False, inner='box')


In [ ]:
min_features = 1000

epi.pp.coverage_cells(adata_atac, binary=True, log=False, bins=50,
               threshold=min_features, save=f'{output_fol}_peaks_coverage_cells.png')
epi.pp.coverage_cells(adata_atac, binary=True, log=10, bins=50,
               threshold=min_features, save=f'{output_fol}_peaks_coverage_cells_log10.png')

In [ ]:
adata_atac

In [ ]:
min_cells = 10
epi.pp.coverage_features(adata_atac, binary=True, log=False, 
                        threshold=min_cells, save=f'{output_fol}_peaks_coverage_peaks.png')
epi.pp.coverage_features(adata_atac, binary=True, log=True, 
                        threshold=min_cells, save=f'{output_fol}_peaks_coverage_peaks_log10.png')

In [ ]:
epi.pp.filter_cells(adata_atac, min_features=min_features)
epi.pp.filter_features(adata_atac, min_cells=min_cells)
adata_atac

In [ ]:
epi.pp.cal_var(adata_atac)

In [ ]:
min_score_value = 0.515
nb_feature_selected = 20000
epi.pl.variability_features(adata_atac,log=None,
                     min_score=min_score_value, nb_features=nb_feature_selected,
                     save=f'{output_fol}/variability_features_plot_peakmatrix.png')

epi.pl.variability_features(adata_atac,log='log10',
                     min_score=min_score_value, nb_features=nb_feature_selected,
                     save=f'{output_fol}/variability_features_plot_peakmatrix_log10.png')

In [ ]:
adata_atac.raw = adata_atac

In [ ]:
# adata_atac = epi.pp.select_var_feature(adata_atac,
#                                   nb_features=nb_feature_selected,
#                                   show=False,
#                                   copy=True)

In [ ]:
adata_atac

In [ ]:
import scanpy.external as sce

In [ ]:
sc.pp.normalize_total(adata_atac, target_sum=1e6)
sc.pp.log1p(adata_atac,base=10)
sc.pp.highly_variable_genes(adata_atac)
sc.pp.pca(adata_atac)
sce.pp.harmony_integrate(adata_atac,key='mouse',max_iter_harmony=20)
sc.pp.neighbors(adata_atac,use_rep='X_pca_harmony')
sc.tl.umap(adata_atac)
sc.pl.umap(adata_atac, color=['log_nb_features', 'celltype'], wspace=0.3)


In [ ]:
adata_atac.var['highly_variable'].value_counts()

In [ ]:
sc.pl.umap(adata_atac, color=['lineage', 'treatment','mouse'], wspace=0.3, save='lineage.png')

In [ ]:
sc.pl.umap(adata_atac, color=['log_nb_features', 'treatment','mouse'], wspace=0.3)

In [ ]:
lin_dict = {}
for lineage in adata_atac.obs['lineage'].cat.categories:
    lin_adata_atac = adata_atac[adata_atac.obs['lineage']==lineage]
    sc.pp.pca(lin_adata_atac)
    sce.pp.harmony_integrate(lin_adata_atac,key='mouse',max_iter_harmony=20)
    sc.pp.neighbors(lin_adata_atac,use_rep='X_pca_harmony')
    sc.tl.leiden(lin_adata_atac)
    sc.tl.umap(lin_adata_atac, min_dist=0.1)
    sc.pl.umap(lin_adata_atac, color=['nb_features', 'celltype'], wspace=0.3, )
    sc.pl.umap(lin_adata_atac, color=['log_nb_features', 'treatment','mouse','celltype'], ncols=2,wspace=0.3,save=f'{lineage}_celltype.png')
    lin_dict[lineage] = lin_adata_atac

In [ ]:
adata_atac.uns['atac']['peak_annotation'].loc['Kdr']

In [ ]:
sc.pl.umap(lin_dict['endothelial'],color = adata_atac.uns['atac']['peak_annotation'].loc['Kdr']['peak'].values)

In [ ]:
sc.pl.dotplot(lin_dict['endothelial'],
              adata_atac.uns['atac']['peak_annotation'].loc['Edn1']['peak'].values,
             groupby=['celltype','treatment'])

In [ ]:
sc.pl.dotplot(lin_dict['endothelial'],
              adata_atac.uns['atac']['peak_annotation'].loc['Cdkn1a']['peak'].values,
             groupby=['celltype','treatment'],
             save='endo_cdkn1a_atac.png')

In [ ]:
sc.pl.dotplot(adata_rna[adata_rna.obs['lineage']=='endothelial'],
             'Cdkn1a',
             groupby=['celltype','treatment'],
             save='endo_cdkn1a_rna.png')

In [ ]:
adata_atac.var['tfs'].loc['chr17:29095049-29095880']

In [ ]:
 adata_atac.uns['atac']['peak_annotation'].loc['Cdkn1a']

In [ ]:
 adata_atac.uns['atac']['peak_annotation'].loc['Mki67']

In [ ]:
adata_atac.var.loc['chr8:123884577-123885483']

In [ ]:
sc.pl.dotplot(lin_dict['mesenchymal'],
              adata_atac.uns['atac']['peak_annotation'].loc['Acta1']['peak'],
             groupby=['celltype'])

In [ ]:
sc.pl.dotplot(adata_atac,
              adata_atac.uns['atac']['peak_annotation'].loc['Mki67']['peak'],
              title='Mki67 peaks',
             groupby=['celltype', 'treatment'],
             save='mki67_peaks.png')